In [4]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pickle as pkl
import sys

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *

import torch.utils.data as data
import torch
from torch_geometric.data import Data
from torch.nn import Linear
from torch_geometric.nn import GCNConv
from torch.utils.data import Dataset

import os
import torch
import torch.nn.functional as F
from tqdm import tqdm
from torch_geometric.loader import NeighborLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.nn import MessagePassing, SAGEConv
from ogb.nodeproppred import Evaluator #PygNodePropPredDatase

from IPython.display import clear_output
import wandb

In [3]:
# Read files
path = "C:/Users/gsamp/OneDrive/Documents/AI-3/2n Semestre/Projecte de Síntesi 2/GraphAnomaly/dades_guillem/"
df_train_init = pd.read_csv(path + "train_set.csv") 
df_test_init = pd.read_csv(path + "test_set.csv")


path = "C:/Users/gsamp/OneDrive/Documents/AI-3/2n Semestre/Projecte de Síntesi 2/GraphAnomaly/dades_guillem/"
df_classes = pd.read_csv(path + "elliptic_txs_classes.csv") # Nodes' labels
df_edges_init = pd.read_csv(path + "elliptic_txs_edgelist.csv") # Edges
df_features = pd.read_csv(path + "elliptic_txs_features.csv", header=None) # Nodes' features

In [4]:
# AIXO JA NO HAURIA DE CALDRE, ESTA AL DATA SPLIT
# path = "C:/Users/gsamp/OneDrive/Documents/AI-3/2n Semestre/Projecte de Síntesi 2/GraphAnomaly/dades_guillem/"
# df_classes = pd.read_csv(path + "elliptic_txs_classes.csv") # Nodes' labels
# df_edges_init = pd.read_csv(path + "elliptic_txs_edgelist.csv") # Edges
# df_features = pd.read_csv(path + "elliptic_txs_features.csv", header=None) # Nodes' features

# Change column names of df_features
colNames1 = {'0': 'txId', 1: "Time step"}
colNames2 = {str(ii+2): "Local_feature_" + str(ii+1) for ii in range(93)}
colNames3 = {str(ii+95): "Aggregate_feature_" + str(ii+1) for ii in range(72)}

colNames = dict(colNames1, **colNames2, **colNames3 )
colNames = {int(jj): item_kk for jj,item_kk in colNames.items()}

df_features = df_features.rename(columns=colNames)

In [13]:
# AIXO JA NO HAURIA DE CALDRE, ESTA AL DATA SPLIT
def prep_df(feats: pd.DataFrame, edges: pd.DataFrame):
    #1 és la classe illicit, 2 la  licit
    df_feats = feats.loc[feats['class'].isin([1,2,3])]
    df_feats['class'] = df_feats['class'].replace({1: 0, 2: 1})
    df_feats = df_feats.reset_index(drop=True)

    #ens quedem només amb els edges que apareixen en el nodes d'entrenament
    df_edges = edges.loc[((edges['txId1'].isin(df_feats['txId'])) & (df_edges_init['txId2'].isin(df_feats['txId'])))]
    df_edges = df_edges.reset_index(drop=True)
    print(f"contador de valors per classe: \n {df_feats['class'].value_counts()}\n")
    return  df_feats, df_edges

# df_train, df_edges_train = prep_df(df_train_init, df_edges_init)
# df_test, df_edges_test = prep_df(df_test_init, df_edges_init)
df_unk, df_edges_unk = prep_df(df_train_init, df_edges_init)

contador de valors per classe: 
 class
3    123281
Name: count, dtype: int64



C:\Users\gsamp\AppData\Local\Temp\ipykernel_12228\1010652687.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feats['class'] = df_feats['class'].replace({1: 0, 2: 1})


In [14]:
def map_idx(feats: pd.DataFrame, edges: pd.DataFrame, save = True, loading_dir = "a"):
    mapping_txid = dict(zip(feats['txId'], list(feats.index)))
    dir = 'dades_guillem/' + str(loading_dir) + '.pkl'
    if save:
        df_edges_mapped = edges.replace({'txId1': mapping_txid, 'txId2': mapping_txid})
        
        df_edges_mapped.to_pickle(loading_dir)
    else:
        df_edges_mapped = pd.read_pickle(loading_dir)
    return df_edges_mapped

# df_edges_mapped_train = map_idx(feats = df_train, edges = df_edges_train, save = True, loading_dir='train')
# df_edges_mapped_test = map_idx(feats = df_test, edges = df_edges_test, save = True, loading_dir='test') # Això no hauria de ser test?
df_edges_mapped_unk = map_idx(feats = df_unk, edges = df_edges_unk, save = True, loading_dir='unk')


In [17]:
def get_data(feats: pd.DataFrame, edges:pd.DataFrame):
    x = torch.tensor(feats.drop(columns=['class', 'Time Step', 'txId']).values, dtype=torch.float)
    edge_index = torch.tensor(edges.values, dtype=torch.long).T
    y = torch.tensor(feats['class'].values)
    time = torch.tensor(feats['Time Step'].values)
    data = Data(x=x, edge_index=edge_index, y=y, time=time)
    return data


# train_data = get_data(df_train, df_edges_mapped_train)
# test_data = get_data(df_test, df_edges_mapped_test)
unk_data = get_data(df_unk, df_edges_mapped_unk)

In [15]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels,
                 hidden_channels, out_channels,
                 n_layers=2):
        
        super(SAGE, self).__init__()
        self.n_layers = n_layers
        self.layers = torch.nn.ModuleList()
        self.layers_bn = torch.nn.ModuleList()
        if n_layers == 1:
            self.layers.append(SAGEConv(in_channels, out_channels,   normalize=False))
        elif n_layers == 2:
            self.layers.append(SAGEConv(in_channels, hidden_channels, normalize=False))
            self.layers_bn.append(torch.nn.BatchNorm1d(hidden_channels))
            # self.layers.append(SAGEConv(hidden_channels, out_channels, normalize=False))
        else:
            self.layers.append(SAGEConv(in_channels, hidden_channels, normalize=False))
            # self.layers_bn.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(n_layers - 2):
            self.layers.append(SAGEConv(hidden_channels,  hidden_channels, normalize=False))
            self.layers_bn.append(torch.nn.BatchNorm1d(hidden_channels))
                    
                
        if n_layers != 1:
            self.layers.append(SAGEConv(hidden_channels, out_channels, normalize=False))
        
        for layer in self.layers:
            layer.reset_parameters()
            
            
            
    def forward(self, x, edge_index):
        if len(self.layers) > 1:
            looper = self.layers[:-1]
        else:
            looper = self.layers
        
        for i, layer in enumerate(looper):
            x = layer(x, edge_index)
            # print(f"SHAPE: {x.shape}, step: {i}")
            # print(f"Step: {i}")
            try:
                x = self.layers_bn[i](x)
            except Exception as e:
                abs(1)
            finally:
                x = F.relu(x)
                x = F.dropout(x, p=0.5, training=self.training)
        
        if len(self.layers) > 1:
            x = self.layers[-1](x, edge_index)
        return F.log_softmax(x, dim=-1), torch.var(x)
    
    def inference(self, total_loader, device):
        xs = []
        var_ = []
        for batch in total_loader:
            out, var = self.forward(batch.x.to(device), batch.edge_index.to(device))
            out = out[:batch.batch_size]
            xs.append(out.cpu())
            var_.append(var.item())
        
        out_all = torch.cat(xs, dim=0)
        
        return out_all, var_

Entrenar models

In [8]:
EPOCHS = 101
layers_list = [2]
# layers_list = [4]
wb = True
for LAYERS in layers_list:
    model = SAGE(train_data.x.shape[1], 256, torch.unique(train_data.y).size(0), n_layers=LAYERS)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.03)
    scheduler = ReduceLROnPlateau(optimizer, 'max', patience=7)
    criterion = torch.nn.CrossEntropyLoss()
    if wb:
        wandb.init(
            # set the wandb project where this run will be logged
            project="GraphAnomaly",
            name = f"2_SAGE - n_layers: {LAYERS}",
            # track hyperparameters and run metadata
            config={
            "architecture": "SAGE_3",
            "dataset": "Time Steps elliptic",
            "epochs": EPOCHS,
            "layers": LAYERS
            }
        )

    def train(data, epoch):
        optimizer.zero_grad()  # Clear gradients.
        out, h = model(data.x, data.edge_index)  # Perform a single forward pass.
        pred = out.argmax(dim=1)
        loss = criterion(out, data.y)  # Compute the loss solely based on the training nodes.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        
        # Calculate accuracy
        correct = pred.eq(data.y).sum().item()
        total = len(data.y)
        accuracy = correct / total
        
        if wb:
            wandb.log({
                "epoch": epoch,
                "train_loss": loss,
                "train_accuracy": accuracy,
            })
        if epoch%5 == 0:
            max_value = torch.max(pred)
            counts = torch.bincount(pred, minlength=max_value.item() + 1)
            print(f"""Nodes classified as fraud: {counts[0]}, 
                nodes classified as licit: {counts[1]}""")
            
            
        return loss, accuracy

    def test(data, epoch):
        model.eval()  # Set the model to evaluation mode.
        with torch.no_grad():
            out, _ = model(data.x, data.edge_index)  # Perform a forward pass.
            loss = criterion(out, data.y)
            
            pred = out.argmax(dim=1)  # Get the predicted labels by selecting the class with the highest probability.
            # Calculate accuracy
            correct = pred.eq(data.y).sum().item()
            total = len(data.y)
            accuracy = correct / total
            
            #precision, recall, f1 score
            precision = precision_score(data.y.cpu().numpy(), pred.cpu().numpy(), average=None)
            recall = recall_score(data.y.cpu().numpy(), pred.cpu().numpy(), average=None)
            f1 = f1_score(data.y.cpu().numpy(), pred.cpu().numpy(), average=None)
            
            other_metrics = [precision, recall, f1]
            if wb:
                wandb.log({
                "epoch": epoch,
                "test_loss": loss,
                "test_accuracy": accuracy,
                "test_precision_class_0": precision[0],
                "test_precision_class_1": precision[1],
                "test_recall_class_0": recall[0],
                "test_recall_class_1": recall[1],
                "test_f1_score_class_0": f1[0],
                "test_f1_score_class_1": f1[1]
            })
            
        return loss, accuracy, other_metrics
            
        
        
    train_loss_history = []
    test_loss_history = []
    train_acc_hist = []
    test_acc_hist = []

    for epoch in range(EPOCHS):
        train_loss, train_acc = train(train_data, epoch)
        train_loss_history.append(train_loss.detach().numpy())
        train_acc_hist.append(train_acc)
        
        test_loss, test_acc, metrics = test(test_data, epoch)
        test_loss_history.append(test_loss.detach().numpy())
        test_acc_hist.append(test_acc)
        
        if epoch%5 == 0:
            print(f"""Epoch: {epoch}, 
            Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc:.4f}
            Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}
            """)
            
            print(f"Precision: {metrics[0]}, Recall: {metrics[1]}, F1 Score: {metrics[2]} \n")
            print("---------------------------------------------------------------------------------")
    
    
        

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gsamper. Use `wandb login --relogin` to force relogin


Nodes classified as fraud: 20593, 
                nodes classified as licit: 16733
Epoch: 0, 
            Training Loss: 0.9328, Training Accuracy: 0.4679
            Test Loss: 1.0730, Test Accuracy: 0.7973
            
Precision: [0.         0.79725049], Recall: [0. 1.], F1 Score: [0.         0.88718906] 

---------------------------------------------------------------------------------


c:\Users\gsamp\anaconda3\envs\visionEnv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\gsamp\anaconda3\envs\visionEnv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\gsamp\anaconda3\envs\visionEnv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

Nodes classified as fraud: 5, 
                nodes classified as licit: 37321
Epoch: 5, 
            Training Loss: 0.4026, Training Accuracy: 0.9284
            Test Loss: 1.0265, Test Accuracy: 0.8538
            
Precision: [0.79726651 0.859689  ], Recall: [0.37373198 0.97583164], F1 Score: [0.50890585 0.91408585] 

---------------------------------------------------------------------------------
Nodes classified as fraud: 2380, 
                nodes classified as licit: 34946
Epoch: 10, 
            Training Loss: 0.2496, Training Accuracy: 0.9550
            Test Loss: 0.7618, Test Accuracy: 0.9132
            
Precision: [0.85440106 0.92467969], Recall: [0.68926855 0.97012899], F1 Score: [0.76300236 0.94685926] 

---------------------------------------------------------------------------------
Nodes classified as fraud: 2491, 
                nodes classified as licit: 34835
Epoch: 15, 
            Training Loss: 0.1601, Training Accuracy: 0.9646
            Test Loss: 0.5524,

RuntimeError: Parent directory GNN_models/trained_models does not exist.

In [11]:
save_model = False
if save_model:
    torch.save(model.state_dict(), 'final_model_2_layers.pth')

Load model

In [23]:
unk_data.x.shape[1]

165

In [35]:
# Instantiate the model
model = SAGE(unk_data.x.shape[1], 256, 2, n_layers=2)

# Load the model's state dict
model.load_state_dict(torch.load('C:/Users/gsamp/OneDrive/Documents/AI-3/2n Semestre/Projecte de Síntesi 2/GraphAnomaly/GNN_models/trained_models/final_model_2_layers.pth'))

# Set the model to evaluation mode
model.eval()

with torch.no_grad():
    # Forward pass through the model
    output = model(unk_data.x, unk_data.edge_index)

# 'output' now contains the model's predictions for the unseen data
output[0]
predicted_classes = torch.argmax(output[0], dim=1)

# Count the occurrences of each value
value_counts = torch.bincount(predicted_classes)

# Print the distribution
for value, count in enumerate(value_counts):
    print(f"Value {value}: {count.item()} occurrences")

Value 0: 7341 occurrences
Value 1: 115940 occurrences
